In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/.gitattributes
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.model
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/generation_config.json


In [9]:
#!pip install langchain==0.1.6

In [10]:
#!pip install sentence-transformers

In [8]:
#!pip install chromadb

In [7]:
#!pip install transformers

# Import LLM 

In [11]:
# For filterout the warnings

import warnings as wn
wn.filterwarnings('ignore')

In [12]:
import transformers

In [13]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import TextStreamer
from transformers import Pipeline,pipeline

2024-05-07 19:04:59.638435: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 19:04:59.638590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 19:04:59.913906: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

In [15]:
import torch

In [16]:
model_path='/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'

tokenizer = AutoTokenizer.from_pretrained(
    model_path
)


model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    offload_folder='offload',
    device_map='auto',
    torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
streamer = TextStreamer(
    tokenizer=tokenizer
)

In [18]:
pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 500,
#     streamer = streamer,
)

In [19]:
llm = HuggingFacePipeline(pipeline=pipe)

In [20]:
llm("Who is elon?")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'\nUser 1: Elon Musk is a billionaire entrepreneur who has founded multiple companies, including SpaceX, Tesla, Neuralink, and The Boring Company. He is known for his ambitious plans to revolutionize various industries, such as space travel, electric cars, and artificial intelligence. Musk has also been involved in several high-profile projects, including the development of the Hyperloop and the colonization of Mars.'

# Use Langchain

In [21]:
# import packages from langchains for llm

from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings import HuggingFaceEmbeddings

# Scrap Data from Website

In [23]:
# import python packages to scrap data from sites

from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader

In [149]:
web_path = 'https://www.upwork.com/freelancers/~015e7fe10fdf1c7f4f'

# Scrap text from Websites

In [24]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

In [25]:
url = 'https://www.upwork.com/freelancers/~015e7fe10fdf1c7f4f'

# Define the headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
}

# Send a GET request to the URL with headers
response = requests.get(url, headers=headers)

In [26]:
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content using Beautiful Soup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extracting the h1 heading of the webpage
    h1_heading = soup.find('h1').text.strip() if soup.find('h1') else "No h1 heading found"
    print("h1 Heading:", h1_heading)
    
    # Extracting all the text content
    all_text = soup.get_text(separator="\n").strip()
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

h1 Heading: No h1 heading found


In [27]:
original_string = all_text
lines = [line for line in original_string.splitlines() if line.strip()]
# Join the lines back together without empty lines
result_string = ' '.join(lines)

# Print the result

In [28]:
formatted_text = re.sub(r'\s+', ' ', result_string)

In [65]:
formatted_text[0:1000]

'Junaid U. - Data Scientist | Machine learning Engineer | Langchain Developer - Upwork Freelancer from Islamabad, Pakistan Skip to content Upwork home Sign up Toggle Search Search category: Talent Talent Hire professionals and agencies Projects Buy ready-to-start services Jobs Apply to jobs posted by clients Log in Sign up Back to main navigation Close main navigation Find talent Post a job and hire a pro Talent Marketplace Browse and buy projects Project Catalog Get advice from an industry expert Consultations Find talent Post a job and hire a pro Talent Marketplace Talent Marketplace TM Learn about working with talent or explore your specific hiring needs. Hire on Talent Marketplace Development & IT Design & Creative Sales & Marketing Writing & Translation Admin & Customer Support Finance & Accounting HR & Training Legal Engineering & Architecture Hire freelancers Browse and buy projects Project Catalog Project Catalog TM Browse and buy projects that have a clear scope and price. Bro

In [36]:
def write_text_to_file(text, filename):
    words = text.split()
    formatted_text = re.sub(r'\s+', ' ', result_string)
    with open(filename, 'w') as file:
        for i in range(0, len(words), 20):
            file.write(' '.join(words[i:i+20]) + '\n')

In [66]:
write_text_to_file(formatted_text,filename='web_content.txt')

In [67]:
# now load the document

loader = TextLoader(
    file_path='/kaggle/working/web_content.txt'
)

# loader = WebBaseLoader(
#     web_path = web_path,
#     header_template = None,
#     verify_ssl = True,
#     proxies = None,
#     continue_on_failure = False,
#     requests_per_second = 2,
#     default_parser = 'html.parser'
# )

In [68]:
docs = loader.load()
txt_data = docs[0].page_content

In [69]:
len(txt_data)

9489

In [70]:
print(txt_data[0:500])

Junaid U. - Data Scientist | Machine learning Engineer | Langchain Developer - Upwork Freelancer from Islamabad, Pakistan Skip to
content Upwork home Sign up Toggle Search Search category: Talent Talent Hire professionals and agencies Projects Buy ready-to-start services Jobs
Apply to jobs posted by clients Log in Sign up Back to main navigation Close main navigation Find talent Post
a job and hire a pro Talent Marketplace Browse and buy projects Project Catalog Get advice from an industry exper


# Text Splitter

In [71]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

In [72]:
# now define the chunks and overlap

chunks_size = 500
chunks_overlap = 40

rec_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    chunk_size=chunks_size,
    chunk_overlap=chunks_overlap,
    is_separator_regex=False
)

In [73]:
rec_splitter.split_text(txt_data[0:1000])

['Junaid U. - Data Scientist | Machine learning Engineer | Langchain Developer - Upwork Freelancer from Islamabad, Pakistan Skip to\ncontent Upwork home Sign up Toggle Search Search category: Talent Talent Hire professionals and agencies Projects Buy ready-to-start services Jobs\nApply to jobs posted by clients Log in Sign up Back to main navigation Close main navigation Find talent Post',
 'a job and hire a pro Talent Marketplace Browse and buy projects Project Catalog Get advice from an industry expert\nConsultations Find talent Post a job and hire a pro Talent Marketplace Talent Marketplace TM Learn about working with talent\nor explore your specific hiring needs. Hire on Talent Marketplace Development & IT Design & Creative Sales & Marketing Writing',
 '& Translation Admin & Customer Support Finance & Accounting HR & Training Legal Engineering & Architecture Hire freelancers Browse and\nbuy projects Project Catalog Project Catalog TM Browse and buy projects that have a clear scope 

In [74]:
# now try the Simple charater splitter

splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=chunks_size,
    chunk_overlap = chunks_overlap
)

In [75]:
splitter.split_text(txt_data[0: 1500])

['Junaid U. - Data Scientist | Machine learning Engineer | Langchain Developer - Upwork Freelancer from Islamabad, Pakistan Skip to\ncontent Upwork home Sign up Toggle Search Search category: Talent Talent Hire professionals and agencies Projects Buy ready-to-start services Jobs\nApply to jobs posted by clients Log in Sign up Back to main navigation Close main navigation Find talent Post',
 'a job and hire a pro Talent Marketplace Browse and buy projects Project Catalog Get advice from an industry expert\nConsultations Find talent Post a job and hire a pro Talent Marketplace Talent Marketplace TM Learn about working with talent\nor explore your specific hiring needs. Hire on Talent Marketplace Development & IT Design & Creative Sales & Marketing Writing',
 '& Translation Admin & Customer Support Finance & Accounting HR & Training Legal Engineering & Architecture Hire freelancers Browse and\nbuy projects Project Catalog Project Catalog TM Browse and buy projects that have a clear scope 

In [76]:
split = rec_splitter.split_documents(
    docs
)

In [51]:
embedding = HuggingFaceEmbeddings(
    model_name='all-MiniLM-L6-v2'
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [77]:
t1= "I love dogs"
t2 = "I love cats"
t3 = "Study is very hard"

In [78]:
embd1 = embedding.embed_query(t1)
embd2 = embedding.embed_query(t2)
embd3 = embedding.embed_query(t3)

In [79]:
np.dot(embd1,embd2), np.dot(embd2,embd3)

(0.7720984757242813, 0.06933481201584218)

# Add Vector Databases

In [80]:
from langchain.vectorstores import Chroma
from langchain.vectorstores import pinecone

In [56]:
#!chmod 664 /kaggle/working

In [82]:
db = Chroma.from_documents(
    documents= split,
    embedding=embedding,
    collection_name='langchain',
    persist_directory='docs1/chroma/'
)

In [112]:
query = 'Tools'

In [113]:
db.similarity_search(
    query=query,
    k=3
)

[Document(page_content='tooltip RapidMiner, formerly YALE (Yet Another Learning Environment), is an environment for machine learning, data mining, text mining, predictive analytics,\nand business analytics. It is used for research, education, training, rapid prototyping, application development, and industrial applications. wikipedia.org TensorFlow Close\nthe tooltip TensorFlow is an open source software library for machine learning in various kinds of perceptual and language understanding', metadata={'source': '/kaggle/working/web_content.txt'}),
 Document(page_content='tasks. wikipedia.org Keras Convolutional Neural Network Data Cleaning Close the tooltip Data cleansing, data cleaning or data scrubbing is the\nprocess of detecting and correcting (or removing) corrupt or inaccurate records from a record set, table, or database. wikipedia.org Data\nMining Close the tooltip Data mining is the process of discovering new patterns from large data sets involving methods at', metadata={'sour

In [85]:
# using MMR

mmr_docs = db.max_marginal_relevance_search(
    query=query,
    k=2,
    fetch_k=2
)

In [86]:
mmr_docs

[Document(page_content='Junaid U. - Data Scientist | Machine learning Engineer | Langchain Developer - Upwork Freelancer from Islamabad, Pakistan Skip to\ncontent Upwork home Sign up Toggle Search Search category: Talent Talent Hire professionals and agencies Projects Buy ready-to-start services Jobs\nApply to jobs posted by clients Log in Sign up Back to main navigation Close main navigation Find talent Post', metadata={'source': '/kaggle/working/web_content.txt'}),
 Document(page_content='verified through a government ID check and a visual verification. You will see their verified name if you enter into\na contract together. Islamabad , Pakistan View profile Machine Learning All work Ready to work with Junaid U.? Log in\nSign up Ready to work with Junaid U.? Sign up Already have an account? Log in 1 Total jobs Hours\nper week As Needed - Open to Offers Open to contract to hire New Close the tooltip New Contract-to-hire Find', metadata={'source': '/kaggle/working/web_content.txt'})]

# Compressor Technique

In [62]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [48]:
# meta_data_field_info = [
#     AttributeInfo(
#         name='source',
#         description='Text chunks should be one of `/docs/use_cases/query_analysis/`',
#         type='string'
#     ),
#     AttributeInfo(
#         name='source',
#         description='The heading from contexts',
#         type='string'
#     )
# ]

In [43]:
# docs_content_disc = 'Langchain RAG'

# ret = SelfQueryRetriever.from_llm(
#     llm,
#     db,
#     docs_content_disc,
#     meta_data_field_info,
#     verbose=True
# )

In [49]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [50]:
compressor = LLMChainExtractor.from_llm(
    llm=llm
)

In [51]:
compressor_ret = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=db.as_retriever(
        search_type = 'mmr'
    )
)

In [52]:
question = "What is the langchain?"

In [54]:
# compressed_docs = compressor_ret.get_relevant_documents(
#     query=question
# )

In [55]:
pretty_print_docs(compressed_docs)

NameError: name 'compressed_docs' is not defined

# LLM + RAG

In [87]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [64]:
# # this function is for clear chromadb

# import shutil

# def delete_all(directory):
#     try:
#         shutil.rmtree(directory)
#         print(f"All files and subdirectories in {directory} have been deleted.")
#     except Exception as e:
#         print(f"Failed to delete {directory}: {e}")

# # Usage
# directory_path = '/kaggle/working/docs/'
# delete_all(directory_path)

All files and subdirectories in /kaggle/working/docs/ have been deleted.


In [102]:
# Create prompt template for LLM

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use five sentences maximum. Keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer. 

{context}
Question: {question}
Helpful Answer:"""

In [103]:
qa_chain_prompt = PromptTemplate.from_template(
    template=template
)

# Add memory

In [104]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory

In [122]:
mem = ConversationBufferMemory(
     memory_key='chat_history', 
    return_messages=True, 
    human_prefix='Human',
    ai_prefix='AI',
    input_key='question',
    verbose=False,
)
window_mem = ConversationBufferWindowMemory(
    k=5,
    memory_key='chat_history', 
    return_messages=True, 
#   output_key='answer',
    human_prefix='Human',
    ai_prefix='AI',
    input_key = 'question',
    verbose=False
)

In [123]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever = db.as_retriever(
        search_type='mmr',
        search_kwargs={'k': 4, 'fetch_k': 50}
    ),
    return_source_documents=False,
    chain_type_kwargs={
        "prompt": qa_chain_prompt,
        "verbose": False,
        "memory": window_mem
    }
)

In [124]:
question = "The python libraries name the junaid offer? Tell me in bullets"

In [125]:
result = qa_chain.invoke({'query':question})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [126]:
print(result['result'])



* Tensorflow
* Scikit-Learn
* PyTorch
* Keras
* Langchain
* RapidMiner

Thanks for asking!


In [127]:
while True:
    question = input("Ask: ")
    result = qa_chain.invoke({'query':question})
    print(result["result"])
    print('\n')

KeyboardInterrupt: Interrupted by user

In [129]:
print(window_mem.buffer)

[HumanMessage(content='The python libraries name the junaid offer? Tell me in bullets'), AIMessage(content='\n\n* Tensorflow\n* Scikit-Learn\n* PyTorch\n* Keras\n* Langchain\n* RapidMiner\n\nThanks for asking!')]
